# Assignment 3 - Fine Tuning YOLOv5 for Object Detection


## Overview and Prerequisites

This notebooks contains the conducted object detection experiments.

All used datasets are from https://roboflow.com. In order to download dataset from there, you normally need to register. For convenience, we provide all datasets via a Google Drive. Download them and place the zip files in the `src/yolov5/datasets` directory.

* Oxford Pets (by species): https://drive.google.com/uc?id=1bfyssR7CfzTqJ-JAgaZNuM0lgTumDcZy&export=download
* TODO
* TODO

Install additional dependencies (protobuf with specific version is needed for `TensorBoard`).

In [1]:
!pip3 install protobuf==3.19.4 ipywidgets

If haven't installed PyTorch yet, remove the comment in the following cell and execute it. In case you need a `CUDA` version different from 11.3, please refer to https://pytorch.org/get-started/locally/.

In [2]:
#!pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113

In [3]:
import os

## Constants and Utiltity Functions

In [4]:
datasets_base_path = "datasets" # directory where datasets are located

In [5]:
def adapt_data_yaml_train_val_keys(path):
    """
    Adapts the paths of the train and validation set (keys 'train' and 'val') of a data.yaml file (YOLOv5 format)
    to point to 'path'. The data.yaml file itself also has to be contained in the directory 'path'.
    
    :param str path: Path (directory) where the data.yaml file to be modified is located
    """
    with open(os.path.join(path, "data.yaml"), "r") as f:
        lines = f.readlines()
    
    lines[0] = f"train: {os.path.join(os.pardir, path, 'train', 'images')}\n"
    lines[1] = f"val: {os.path.join(os.pardir, path, 'valid', 'images')}\n"

    with open(os.path.join(path, "data.yaml"), "w") as f:
        f.writelines(lines)

## YOLOv5 - Setup


Clone the YOLOv5 GitHub repository.

In [6]:
!rm -rf yolov5
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 12345, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12345 (delta 2), reused 2 (delta 1), pack-reused 12335
Receiving objects: 100% (12345/12345), 12.17 MiB | 5.65 MiB/s, done.
Resolving deltas: 100% (8516/8516), done.


Install the dependencies of YOLOv5.

In [7]:
!pip3 install -r yolov5/requirements.txt

## Oxford Pets - By Species

* Source: https://public.roboflow.com/object-detection/oxford-pets/2/download/yolov5pytorch
* \# of classes: 2

In [8]:
oxford_pets_species_zip_filename = "oxford_pets_species.zip"
oxford_pets_species_extract_dir = "oxford_pets_species"

Extract the zip file to the specified target directory.

In [9]:
oxford_pets_species_zip_path = os.path.join(datasets_base_path, oxford_pets_species_zip_filename)
oxford_pets_species_extract_path = os.path.join(datasets_base_path, oxford_pets_species_extract_dir)

In [10]:
!rm -rf {oxford_pets_species_extract_path}
!unzip -o {oxford_pets_species_zip_path} -d {oxford_pets_species_extract_path}

Archive:  datasets/oxford_pets_species.zip
 extracting: datasets/oxford_pets_species/README.dataset.txt  
 extracting: datasets/oxford_pets_species/README.roboflow.txt  
 extracting: datasets/oxford_pets_species/data.yaml  
   creating: datasets/oxford_pets_species/test/
   creating: datasets/oxford_pets_species/test/images/
 extracting: datasets/oxford_pets_species/test/images/Abyssinian_127_jpg.rf.4425d22917ead085ad038931b4806c8f.jpg  
 extracting: datasets/oxford_pets_species/test/images/Abyssinian_131_jpg.rf.e8acfb60e4d01529586b9d81930b35a2.jpg  
 extracting: datasets/oxford_pets_species/test/images/Abyssinian_149_jpg.rf.84b8351968b42b38478e0b6c35d58d07.jpg  
 extracting: datasets/oxford_pets_species/test/images/Abyssinian_155_jpg.rf.b0d3e78a69efe3c9aa7bd4416a51e105.jpg  
 extracting: datasets/oxford_pets_species/test/images/Abyssinian_15_jpg.rf.0e12ac0df99238e4f77a9eb02877b769.jpg  
 extracting: datasets/oxford_pets_species/test/images/Abyssinian_167_jpg.rf.de4ef71b17d239eb8920914

 extracting: datasets/oxford_pets_species/test/images/pug_145_jpg.rf.06cdf4e44d76b874717f8d2a99f68665.jpg  
 extracting: datasets/oxford_pets_species/test/images/pug_162_jpg.rf.bf545c42faa5df5d9d0d76e17e48640c.jpg  
 extracting: datasets/oxford_pets_species/test/images/pug_170_jpg.rf.a714c28933c467b7acc611070ff0a75e.jpg  
 extracting: datasets/oxford_pets_species/test/images/pug_176_jpg.rf.73f32c9a08fa120703f1320b492b6b1d.jpg  
 extracting: datasets/oxford_pets_species/test/images/pug_185_jpg.rf.3f19836b96f5d9d63ee39bba2ec541e5.jpg  
 extracting: datasets/oxford_pets_species/test/images/saint_bernard_110_jpg.rf.c91468528ce920012bca25bbd4e7c0a6.jpg  
 extracting: datasets/oxford_pets_species/test/images/saint_bernard_115_jpg.rf.92a9b897d991a817a422bf86ade7fd01.jpg  
 extracting: datasets/oxford_pets_species/test/images/saint_bernard_118_jpg.rf.bddf8e78eba19ff9d2f7cbf3bf90812a.jpg  
 extracting: datasets/oxford_pets_species/test/images/saint_bernard_11_jpg.rf.1a143e0aa200752132aabf086951

 extracting: datasets/oxford_pets_species/train/images/British_Shorthair_128_jpg.rf.6fae8bd4489aa54e136b8223fdeec397.jpg  
 extracting: datasets/oxford_pets_species/train/images/British_Shorthair_129_jpg.rf.b0ddc7b74b9accfb7d185f5e615add06.jpg  
 extracting: datasets/oxford_pets_species/train/images/British_Shorthair_134_jpg.rf.6ec4cd9fc5b82507d75f144744fcf6d7.jpg  
 extracting: datasets/oxford_pets_species/train/images/British_Shorthair_135_jpg.rf.493ddbe62fcffadc1adcc15913af3d8d.jpg  
 extracting: datasets/oxford_pets_species/train/images/British_Shorthair_136_jpg.rf.31e1a02063e150a7e6eb740e9f1159bd.jpg  
 extracting: datasets/oxford_pets_species/train/images/British_Shorthair_137_jpg.rf.2097f2ef3360236aa8b1f7880e835d04.jpg  
 extracting: datasets/oxford_pets_species/train/images/British_Shorthair_139_jpg.rf.3c32014f3f804c95a5712c41fc00aae5.jpg  
 extracting: datasets/oxford_pets_species/train/images/British_Shorthair_140_jpg.rf.ad5f76fa4add27991000089310b75f99.jpg  
 extracting: dat

 extracting: datasets/oxford_pets_species/train/images/Persian_100_jpg.rf.54af43d6c78e330b579a121c0e05053f.jpg  
 extracting: datasets/oxford_pets_species/train/images/Persian_101_jpg.rf.0c4a0b74c24bdb9b1d17f7fd48ec9099.jpg  
 extracting: datasets/oxford_pets_species/train/images/Persian_104_jpg.rf.b75d51c59a7afabbd9e26d4eb4c9f411.jpg  
 extracting: datasets/oxford_pets_species/train/images/Persian_106_jpg.rf.49e45e6b5f2498a983190530fcee2130.jpg  
 extracting: datasets/oxford_pets_species/train/images/Persian_107_jpg.rf.2fc449ee83fd39c0ffdfd2dba47bef7b.jpg  
 extracting: datasets/oxford_pets_species/train/images/Persian_108_jpg.rf.2452f7bef93d5879dfd92c3b16e1c8cc.jpg  
 extracting: datasets/oxford_pets_species/train/images/Persian_111_jpg.rf.33ad67153baff2717a517c435cb806b1.jpg  
 extracting: datasets/oxford_pets_species/train/images/Persian_112_jpg.rf.d72bb8e6327b19c1ba4f84e58cece53c.jpg  
 extracting: datasets/oxford_pets_species/train/images/Persian_114_jpg.rf.db4bf03f63b0d7182f75b0

 extracting: datasets/oxford_pets_species/train/images/american_bulldog_131_jpg.rf.eb7d3d358fa884643b6423384915e0c0.jpg  
 extracting: datasets/oxford_pets_species/train/images/american_bulldog_132_jpg.rf.8368687987c81b80c7c2cdf8dc2816ae.jpg  
 extracting: datasets/oxford_pets_species/train/images/american_bulldog_133_jpg.rf.6210255699b312cd695a556971b66114.jpg  
 extracting: datasets/oxford_pets_species/train/images/american_bulldog_134_jpg.rf.3078a451a4b30afaa1c49f18cc4f833b.jpg  
 extracting: datasets/oxford_pets_species/train/images/american_bulldog_136_jpg.rf.f46b72ef8e913313a00edb00c553a288.jpg  
 extracting: datasets/oxford_pets_species/train/images/american_bulldog_137_jpg.rf.348ad065d50c345e51724ed4ddfb4a0c.jpg  
 extracting: datasets/oxford_pets_species/train/images/american_bulldog_139_jpg.rf.1b6e5476580ae1bcd885d9c75db4c374.jpg  
 extracting: datasets/oxford_pets_species/train/images/american_bulldog_13_jpg.rf.30a10348248c3f0c7c906cc696a69a6e.jpg  
 extracting: datasets/oxf

...

 extracting: datasets/oxford_pets_species/train/images/pomeranian_149_jpg.rf.ac11c44d7ca0105aa782baef69fd5676.jpg  
 extracting: datasets/oxford_pets_species/train/images/pomeranian_14_jpg.rf.775ca139df73e034d6d47df8d2cd7af4.jpg  
 extracting: datasets/oxford_pets_species/train/images/pomeranian_150_jpg.rf.143dc34f653496eabed479c004e83747.jpg  
 extracting: datasets/oxford_pets_species/train/images/pomeranian_151_jpg.rf.14a70699778261c98aff7652c5f02a7c.jpg  
 extracting: datasets/oxford_pets_species/train/images/pomeranian_152_jpg.rf.3f2400f2a252383f99517a66d5ebc475.jpg  
 extracting: datasets/oxford_pets_species/train/images/pomeranian_153_jpg.rf.b28cf21684bb49acc4f39e2689ca8694.jpg  
 extracting: datasets/oxford_pets_species/train/images/pomeranian_154_jpg.rf.2a07fb58ff2ddd446a2e17a330ad6ca0.jpg  
 extracting: datasets/oxford_pets_species/train/images/pomeranian_155_jpg.rf.f2a7593d7cf1d88804201528fcb89fe3.jpg  
 extracting: datasets/oxford_pets_species/train/images/pomeranian_156_jpg

 extracting: datasets/oxford_pets_species/train/images/staffordshire_bull_terrier_110_jpg.rf.32760200c7faccfeb67a7135ba4a82ba.jpg  
 extracting: datasets/oxford_pets_species/train/images/staffordshire_bull_terrier_112_jpg.rf.cc3adf80474a61069749b562495c842c.jpg  
 extracting: datasets/oxford_pets_species/train/images/staffordshire_bull_terrier_113_jpg.rf.68f038a67789760dd81421c3d7bfec01.jpg  
 extracting: datasets/oxford_pets_species/train/images/staffordshire_bull_terrier_116_jpg.rf.d9aa92cb666405d7d115f44be1798f8e.jpg  
 extracting: datasets/oxford_pets_species/train/images/staffordshire_bull_terrier_118_jpg.rf.564dfa1a75cd776b5cf1bed25dcbb1d3.jpg  
 extracting: datasets/oxford_pets_species/train/images/staffordshire_bull_terrier_119_jpg.rf.fb13dc671af50d916874a88f8e50269a.jpg  
 extracting: datasets/oxford_pets_species/train/images/staffordshire_bull_terrier_121_jpg.rf.43f48ba2862b6e565943d41b9a998f82.jpg  
 extracting: datasets/oxford_pets_species/train/images/staffordshire_bull_te

 extracting: datasets/oxford_pets_species/train/labels/english_setter_121_jpg.rf.dbbe8d54955989254417d41852a34918.txt  
 extracting: datasets/oxford_pets_species/train/labels/english_setter_122_jpg.rf.7796975fe4080641ec687ca4d210fa64.txt  
 extracting: datasets/oxford_pets_species/train/labels/english_setter_123_jpg.rf.2ff66cc3fac3315d4b20445b0851a09c.txt  
 extracting: datasets/oxford_pets_species/train/labels/english_setter_124_jpg.rf.9ad7866990adad0cb57088ab0db439f1.txt  
 extracting: datasets/oxford_pets_species/train/labels/english_setter_125_jpg.rf.2bafab399b3722f95a6a59783030e99f.txt  
 extracting: datasets/oxford_pets_species/train/labels/english_setter_126_jpg.rf.47353e702a8801be0f1e061cb6b3415f.txt  
 extracting: datasets/oxford_pets_species/train/labels/english_setter_127_jpg.rf.7cb5af17f2ad0c98a1185ba0e34bf463.txt  
 extracting: datasets/oxford_pets_species/train/labels/english_setter_128_jpg.rf.f411e80df09e588a1174ffc2dde80040.txt  
 extracting: datasets/oxford_pets_specie

 extracting: datasets/oxford_pets_species/valid/images/Siamese_127_jpg.rf.85c5339f4cefaac408d8b902d8aba4ec.jpg  
 extracting: datasets/oxford_pets_species/valid/images/Siamese_151_jpg.rf.4234dbe94b3f4000eb7cb59a2678395c.jpg  
 extracting: datasets/oxford_pets_species/valid/images/Siamese_157_jpg.rf.b93faea16b7025795da979d44c83c000.jpg  
 extracting: datasets/oxford_pets_species/valid/images/Siamese_165_jpg.rf.8e9bd35f5d5e1d32f106052bf53aa76e.jpg  
 extracting: datasets/oxford_pets_species/valid/images/Siamese_167_jpg.rf.aa6bd651f4a954a37534f210a07581f9.jpg  
 extracting: datasets/oxford_pets_species/valid/images/Siamese_168_jpg.rf.89440b068ecd255759d290993c4c8172.jpg  
 extracting: datasets/oxford_pets_species/valid/images/Siamese_175_jpg.rf.718cd6836ab0d9a2a9b5ff66ab48ead9.jpg  
 extracting: datasets/oxford_pets_species/valid/images/Siamese_178_jpg.rf.5e8e2414817e42a68088b2483c555e40.jpg  
 extracting: datasets/oxford_pets_species/valid/images/Siamese_185_jpg.rf.f6e0b34199d8f3111c3910

 extracting: datasets/oxford_pets_species/valid/images/scottish_terrier_112_jpg.rf.07ced135022dbec50f39be15d24f0864.jpg  
 extracting: datasets/oxford_pets_species/valid/images/scottish_terrier_113_jpg.rf.289e9944dd1afa54d503609499c749e4.jpg  
 extracting: datasets/oxford_pets_species/valid/images/scottish_terrier_116_jpg.rf.204b5ac9156f40a9ce5ad1c100f7ca66.jpg  
 extracting: datasets/oxford_pets_species/valid/images/scottish_terrier_136_jpg.rf.b8a9c802fd005b4c48bbe6d84c47fc4d.jpg  
 extracting: datasets/oxford_pets_species/valid/images/scottish_terrier_143_jpg.rf.f0118d87cdc52ada5dd3094771a685ae.jpg  
 extracting: datasets/oxford_pets_species/valid/images/scottish_terrier_145_jpg.rf.379cf8b931143dd6d8aee4854bf46814.jpg  
 extracting: datasets/oxford_pets_species/valid/images/scottish_terrier_151_jpg.rf.e615f0c4c1d9f9855139c74c55c3018c.jpg  
 extracting: datasets/oxford_pets_species/valid/images/scottish_terrier_156_jpg.rf.7e9f72b6fcf16bba3debca3ac9e8504d.jpg  
 extracting: datasets/ox

Adapt the paths in the `data.yaml` to fit our project structure.

In [11]:
adapt_data_yaml_train_val_keys(oxford_pets_species_extract_path)

Fine-tune pre-trained `YOLOv5n`.

In [12]:
weights = "yolov5n.pt"
data = "datasets/oxford_pets_species/data.yaml"
project = "runs/train"
project_name = f"yolov5n_oxford_pets_species"
device = 0 # use first GPU; if no CPU is available, change to "cpu"

!python3 yolov5/train.py --weights {weights} --batch 32 --epochs 150 --data {data} --project {project} --name {project_name} --device {device} --cache

train: weights=yolov5n.pt, cfg=, data=datasets/oxford_pets_species/data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5n_oxford_pets_species, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-270-g6935a54 Python-3.8.10 torch-1.12.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060, 12045MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.828      0.798      0.867      0.484

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     8/149     4.96G    0.0327    0.0144  0.006247        26       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.881      0.855      0.909      0.596

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     9/149     4.96G   0.03167   0.01391  0.006395        33       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.922       0.92      0.956      0.641

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    10/149     4.96G   0.03084   0.01375  0.006308        31       640: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
    33/149     4.96G   0.02485   0.01107  0.003182        31       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.969      0.944      0.983      0.764

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    34/149     4.96G   0.02414    0.0113  0.002904        36       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.958      0.962       0.98      0.761

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    35/149     4.96G    0.0237   0.01097   0.00301        39       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        737      0.968      0.944      0.985      0.766

     Epoch   gpu_mem       box      

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.973       0.96      0.985      0.786

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    59/149     4.96G   0.02134   0.01027  0.002146        33       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738       0.97      0.966      0.986       0.79

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    60/149     4.96G   0.02139  0.009842  0.002072        32       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.966      0.969      0.987      0.789

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    61/149     4.96G   0.02137   0.01015  0.002313        36       640: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
    84/149     4.96G   0.01933  0.009573  0.002067        30       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.977      0.965      0.989      0.798

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    85/149     4.96G   0.01951  0.009502  0.001833        43       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.964      0.976      0.989      0.803

     Epoch   gpu_mem       box       obj       cls    labels  img_size
    86/149     4.96G   0.01911  0.009089  0.001469        36       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738       0.98      0.976      0.992      0.812

     Epoch   gpu_mem       box      

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738       0.99      0.979      0.993      0.832

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   110/149     4.96G   0.01739   0.00887  0.001088        41       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.985      0.983      0.993      0.835

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   111/149     4.96G   0.01687  0.008353  0.001251        41       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.981      0.983      0.993       0.83

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   112/149     4.96G    0.0167  0.008409  0.001154        32       640: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
   135/149     4.96G   0.01596  0.008273  0.001313        35       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.988      0.981      0.994      0.842

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   136/149     4.96G    0.0153  0.008078 0.0006545        33       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.985      0.985      0.994      0.843

     Epoch   gpu_mem       box       obj       cls    labels  img_size
   137/149     4.96G   0.01538  0.008261 0.0008322        32       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        736        738      0.985      0.984      0.994      0.847

     Epoch   gpu_mem       box      

Fine-tune pre-trained `YOLOv5s`.

In [ ]:
weights = "yolov5s.pt"
data = "datasets/oxford_pets_species/data.yaml"
project = "runs/train"
project_name = f"yolov5s_oxford_pets_species"
device = 0 # use first GPU; if no CPU is available, change to "cpu"

!python3 yolov5/train.py --weights {weights} --batch 32 --epochs 150 --data {data} --project {project} --name {project_name} --device {device} --cache